In [6]:
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))

In [7]:
import numpy as np
from timeit import default_timer as timer 
from numba import cuda,jit,vectorize,int32,float32,guvectorize
from math import sqrt
import pickle
import Call_Function as cf

In [9]:
n_user,n_movie,n_rating=cf.read_user_inform()
svd=SVD(n_user,200,n_movie,0,0.1,30)
total_set='./ml-100k/u.data'
svd.read_u_data(cf.data_location)

Read Start
Finish


In [10]:
svd.get_overall_mean()

Time :  2.530684967059642


In [11]:
RMSE_list=[]
start=timer()
print('SVD Start')
for i in range(cf.k_fold):
    svd.gradient_descent(cf.train_set_list[i])
    
    RMSE=svd.predict(test_set_list[i])
    print(RMSE)
    RMSE_list.append(RMSE)
time=timer()-start

print("Time : ",time)

avg_RMSE=sum(RMSE_list)/len(RMSE_list)

print("RMSE : ",avg_RMSE)

if save_or_not:
    save_object("svd_object.p",[svd,RMSE_list,time,avg_RMSE])

SVD Start


KeyboardInterrupt: 

In [6]:
RMSE_list=[]
start=timer()
for i in range(5):
    train_set='./ml-100k/u'+str(i+1)+'.base'
    svd.gradient_descent(train_set)
    test_set='./ml-100k/u'+str(i+1)+'.test'
    RMSE=svd.predict(test_set)
    print(RMSE)
    RMSE_list.append(RMSE)
print("Time : ",timer()-start)
with open("svd_object.p",'wb') as file:
    pickle.dump(svd,file)

0.9615709943027311
0.949974949815643
0.9425239265970955
0.9395484702560938
0.9436179559911538
Time :  6042.349062948953


In [3]:
class SVD:
    #train_set indicate the route of train set
    def __init__(self,n_user,n_factor,n_movie,mean,stdev,epoch,learning_rate=0.002,regulation=0.04,random_state=1):
        self.n_user=n_user
        self.n_factor=n_factor
        self.n_movie=n_movie
        self.rating_matrix=np.zeros((n_user,n_movie),np.int8)
        #default value - 100
        self.overall_mean=0
        self.epochs=epoch
        #default value - 1
        self.random_state=random_state
        self.init_mean=mean
        self.init_stdev=stdev
        self.lr=learning_rate
        self.reg=regulation
        self.bu=None
        self.bi=None
        self.pu=None
        self.qi=None
        
    def read_u_data(self,total_location):
        with open(total_location,'r') as f:
            print("Read Start")
            line=f.readline().split('\t')
            while len(line)>1:
                self.rating_matrix[int(line[0])-1,int(line[1])-1]=int(line[2])
                line=f.readline().split('\t')
            print("Finish")
            
    def get_overall_mean(self):
        start=timer()
        total_count=0
        total_sum=0
        for j in range(self.n_movie):
            for i in range(self.n_user):
                if self.rating_matrix[i,j] != 0:
                    total_count+=1
                    total_sum+=self.rating_matrix[i,j]
        self.overall_mean=total_sum/total_count
        print("Time : ",timer()-start)
        
        
    def predict(self,test_set):
        test_count=0
        r_sum=0
        with open(test_set,'r') as file:
            line=file.readline().split('\t')
            
            while len(line) >1:
                test_count+=1
                movie_index=int(line[1])-1
                user_index=int(line[0])-1
                rating=int(line[2])
                product=0
                for f in range(self.n_factor):
                    product+=self.qi[movie_index,f]*self.pu[user_index,f]
                error=rating-(self.overall_mean+self.bu[user_index]+self.bi[movie_index]+product)
                r_sum+=(error)**2
                line=file.readline().split('\t')
        return sqrt(r_sum/test_count)
    
    
    def gradient_descent(self,train_set):
        SVD.init_vector(self)
        for i in range(self.epochs):
            with open(train_set,'r') as file:
                line=file.readline().split('\t')
                while len(line) >1:
                    #print("Loading {}\r".format(loading), end='')
                    #number of movie id relating to train_set
                    movie_index=int(line[1])-1
                    user_index=int(line[0])-1
                    rating=int(line[2])
                    product=0
                    for f in range(self.n_factor):
                        product+=self.qi[movie_index,f]*self.pu[user_index,f]
                    error=rating-(self.overall_mean+self.bu[user_index]+self.bi[movie_index]+product)
                    self.bu[user_index]+=self.lr*(error-self.reg*self.bu[user_index])
                    self.bi[movie_index]+=self.lr*(error-self.reg*self.bi[movie_index])
                    for i in range(self.n_factor):
                        pu_=self.pu[user_index,i]
                        qi_=self.qi[movie_index,i]
                        self.pu[user_index,i]+=self.lr*(error*qi_-self.reg*pu_)
                        self.qi[movie_index,i]+=self.lr*(error*pu_-self.reg*qi_)
                    line=file.readline().split('\t')
    #b_u와 b_i를 initialize한다.
    def init_vector(self):
        generator=np.random.RandomState(self.random_state)
        self.bu=np.zeros(self.n_user,np.double)
        self.bi=np.zeros(self.n_movie,np.double)
        self.pu=generator.normal(self.init_mean,self.init_stdev,(self.n_user,self.n_factor))
        self.qi=generator.normal(self.init_mean,self.init_stdev,(self.n_movie,self.n_factor))
        
                    
  